## 🔐 Add Widevine Keys (Optional)

**How to get keys:**
1. Install [Widevine L3 Decrypter](https://addons.mozilla.org/en-US/firefox/addon/widevine-l3-decrypter/) on Firefox
2. Play any video from the course
3. Open extension → Click "Guess" → Copy KID:KEY
4. Paste in the code cell below (one per line)

## Step 1️⃣ Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
if not os.path.isdir('/content/drive/MyDrive'):
    drive.mount('/content/drive')
    print("✅ Google Drive mounted!")
else:
    print("✅ Google Drive already mounted")

# Create output directory
OUTPUT_DIR = '/content/drive/MyDrive/Udemy_Courses'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📁 Downloads will go to: {OUTPUT_DIR}")

## Step 2️⃣ Install System Dependencies

In [ ]:
%%bash
echo "📦 Installing dependencies..."

# FFmpeg
apt-get install -y -qq ffmpeg > /dev/null 2>&1

# aria2
apt-get install -y -qq aria2 > /dev/null 2>&1

# shaka-packager
wget -q https://github.com/shaka-project/shaka-packager/releases/download/v2.6.1/packager-linux-x64 -O /usr/local/bin/shaka-packager 2>/dev/null
chmod +x /usr/local/bin/shaka-packager

echo "✅ System dependencies installed!"

## Step 3️⃣ Clone Repository & Install Python Packages

In [ ]:
import os

# Clone repository
REPO_URL = "https://github.com/PerspicaciousGuy/Advance-Udemy-Downloader.git"
DEST_DIR = "/content/udemy-downloader"

if not os.path.exists(DEST_DIR):
    print("📥 Cloning repository...")
    os.system(f"git clone {REPO_URL} {DEST_DIR} > /dev/null 2>&1")
else:
    print("✅ Repository already exists")

os.chdir(DEST_DIR)

# Install Python packages
print("📦 Installing Python packages...")
!pip install -q requests beautifulsoup4 m3u8 browser-cookie3 pysrt webvtt-py coloredlogs python-dotenv pathvalidate tqdm lxml yt-dlp demoji bitstring protobuf > /dev/null 2>&1

print("✅ All dependencies installed!")

## Step 4️⃣ Configure & Download

### 🔑 Add Your Cookies Here

**How to get cookies:**
1. Install [Cookie-Editor](https://cookie-editor.com/) extension
2. Log in to Udemy in your browser
3. Open Cookie-Editor → Export as **Netscape** format
4. Paste between the `"""` marks below

### 🔗 Add Your Course URL

Find it in your browser address bar when viewing a Udemy course

---

In [ ]:
# ========== EDIT THESE ==========

# OPTION 1: Use Bearer Token (Recommended - more reliable than cookies)
USE_BEARER_TOKEN = True
BEARER_TOKEN = ""  # Paste your bearer token here (or set USE_BEARER_TOKEN = False to use cookies)

# OPTION 2: Use Cookies (if bearer token fails)
COOKIES = """
# Paste cookies here
"""

# Paste Widevine keys here (optional - leave empty to skip DRM)
# Format: KID:KEY (one per line, separated by colons)
# Example: d989f2607309455abe7a900ecd8d1994:603d9dbb4c9797b76a49c3ad4c203558
WIDEVINE_KEYS = """
"""

# Your Udemy course URL
COURSE_URL = "https://www.udemy.com/course/YOUR-COURSE-URL/learn/"

# Download options (edit if needed)
VIDEO_QUALITY = "720"  # "720", "1080", or "" for best
DOWNLOAD_ASSETS = True
DOWNLOAD_QUIZZES = True
DOWNLOAD_CAPTIONS = True
CONCURRENT_DOWNLOADS = 15

# ================================

import os
import json
import subprocess

os.chdir('/content/udemy-downloader')

# Handle authentication
if USE_BEARER_TOKEN:
    if BEARER_TOKEN.strip():
        print("✅ Using Bearer Token authentication")
        auth_method = "bearer_token"
        auth_value = BEARER_TOKEN.strip()
    else:
        print("❌ ERROR: Bearer token is empty! Set USE_BEARER_TOKEN = False to use cookies instead.")
        exit()
else:
    if COOKIES.strip():
        print("✅ Using Cookies authentication")
        with open('cookies.txt', 'w') as f:
            f.write(COOKIES.strip())
        auth_method = "cookies"
    else:
        print("❌ ERROR: Please paste your cookies above!")
        exit()

# Create keyfile with Widevine keys (if provided)
keys_dict = {}
if WIDEVINE_KEYS.strip():
    for line in WIDEVINE_KEYS.strip().split('\n'):
        line = line.strip()
        if ':' in line and line:
            kid, key = line.split(':', 1)
            keys_dict[kid.strip()] = key.strip()
    print(f"✅ Loaded {len(keys_dict)} Widevine key(s)")
else:
    print("⏭️ No Widevine keys provided (will skip DRM videos)")

with open('keyfile.json', 'w') as f:
    json.dump(keys_dict, f)

# Verify
print(f"📌 Course: {COURSE_URL}")
print(f"📁 Output: {OUTPUT_DIR}")
print(f"🎥 Quality: {VIDEO_QUALITY}p")
print("")
print("✅ Configuration ready! Running download...")
print("="*60)

# Build command
cmd = [
    "python", "main.py",
    "-c", COURSE_URL,
    "-o", OUTPUT_DIR,
    "-cd", str(CONCURRENT_DOWNLOADS)
]

# Add authentication method
if auth_method == "bearer_token":
    cmd.extend(["-b", auth_value])
else:
    cmd.extend(["--browser", "file"])

if VIDEO_QUALITY:
    cmd.extend(["-q", VIDEO_QUALITY])
if DOWNLOAD_ASSETS:
    cmd.append("--download-assets")
if DOWNLOAD_QUIZZES:
    cmd.append("--download-quizzes")
if DOWNLOAD_CAPTIONS:
    cmd.append("--download-captions")

# Run download
try:
    result = subprocess.run(cmd, check=True, capture_output=True, text=True)
    print(result.stdout)
    print("="*60)
    print("✅ Download completed!")
    print(f"📁 Check Google Drive: {OUTPUT_DIR}")
except subprocess.CalledProcessError as e:
    print("❌ Error occurred:")
    print(e.stderr)
    print("\n💡 Common fixes:")
    print("  1. If using bearer token: Make sure it's valid and not expired")
    print("  2. If using cookies: Re-export fresh cookies from browser")
    print("  3. Verify the course URL is correct")
    print("  4. Check if you have access to the course")
    print("  5. Add Widevine keys if course has DRM protection")

---

## ❓ Troubleshooting

**"No bearer token" error?**
- Your cookies are invalid or expired
- Export fresh cookies from your browser

**"Course not found"?**
- Verify the course URL is correct
- Make sure you have access to the course

**Downloads not appearing in Drive?**
- Wait a few minutes for sync
- Check `Udemy_Courses` folder in your Drive

**Session expired?**
- Colab sessions timeout after ~12 hours
- Restart and re-run all cells

---

## ⚠️ Legal Notice

This tool is for **educational purposes only**. Downloading courses may violate Udemy's Terms of Service. Use at your own risk.